# Installing

In [ ]:
!git clone https://github.com/salesforce/LAVIS.git


In [ ]:
!rm -rf 'LAVIS/lavis/projects/blip2'
!cp -r 'blip2' 'LAVIS/lavis/projects/blip2'

In [ ]:
old_code = "image_ids = samples[\"image_id\"].view(-1,1)"
new_code = "image_ids = torch.tensor([int(x.split('_')[1]) for x in samples[\"image_id\"]]).view(-1, 1).to(image.device)"

def update_file(file_path, old, new):
    with open(file_path, 'r') as file:
        content = file.read()
    content = content.replace(old, new)
    with open(file_path, 'w') as file:
        file.write(content)
update_file('LAVIS/lavis/runners/runner_base.py', old_code, new_code)
update_file('LAVIS/lavis/models/blip2_models/blip2_qformer.py', old_code, new_code)

In [ ]:
!cd LAVIS && pip install .

# Download data

In [ ]:
!python LAVIS/lavis/datasets/download_scripts/download_coco.py

# Evaluation
### Evaluating Pre-trained Models on Task Datasets

In [ ]:
!cd LAVIS && python -m torch.distributed.run --nproc_per_node=1 evaluate.py --cfg-path lavis/projects/blip2/eval/caption_coco_opt2.7b_eval.yaml


# Training

### Pretrain stage 1

In [ ]:
!cd LAVIS && python -m torch.distributed.run --nproc_per_node=1 train.py --cfg-path lavis/projects/blip2/train/pretrain_stage1.yaml

### Pretrain stage 2

In [ ]:
!cd LAVIS && python -m torch.distributed.run --nproc_per_node=1 train.py --cfg-path lavis/projects/blip2/train/pretrain_stage2.yaml

### Finetuning on COCO caption dataset

In [ ]:
!cd LAVIS && python -m torch.distributed.run --nproc_per_node=1 train.py --cfg-path lavis/projects/blip2/train/caption_coco_ft.yaml

### Finetuning on COCO retrieval dataset

In [ ]:
!cd LAVIS && python -m torch.distributed.run --nproc_per_node=1 train.py --cfg-path lavis/projects/blip2/train/retrieval_coco_ft.yaml